<a href="https://colab.research.google.com/github/Claudiars20/Twitter-NLP/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librerías:**
---

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json
import random
import os
import pandas as pd
import tweepy
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import drive

# **LDA (Latent Dirichlet Allocation)**
---

Leemos data de trends preprocesada:

In [2]:
df = pd.read_csv("TRENDS-CSV.csv")
text_data = []
for i in range(df.shape[0]):
  a = df.iloc[i,2][2:-2]
  a = a.split("', '")
  text_data.append(a)
print(text_data)

[['mismo', 'noalcomunismo', 'favor', 'cojudigno', 'arruinar', 'vigilar', 'malograr', 'sacar', 'solución', 'patria', 'vigilatukarpa', 'final', 'idioto', 'esperar', 'innecesario', 'narcotráfico', 'equilibrio', 'diablo', 'premier', 'nuevo', 'ciudadano', 'resultado', 'idiota', 'lapicito', 'filoterruco', 'ahora', 'chiste', 'opción', 'detrás', 'quito', 'marcha', 'sumar', 'poder', 'gobernar', 'nariz', 'cantado', 'hacer', 'dejenlar', 'aburrir', 'interés', 'mil', 'dar', 'beneficio', 'culpable', 'asumetuvoto', 'sorprender', 'exigir', 'jajajjajajaj', 'hashtag', 'red', 'jajajaja', 'voto', 'castillopresidenteilegitimo', 'tonto', 'empezar', 'afinar', 'unir', 'dejar', '1ºministro', 'varios', 'buen', 'entonces', 'desear', 'empanada', 'pasar', 'demás', 'caviar', 'largar', 'tirar', 'traidor', 'agarrar', 'vacancia', 'congreso', 'demostrar', 'dividir', 'vender', 'defensa', 'constitución', 'pensar', 'grado', 'sino', 'canjeable', 'perdisye', 'gabinar', 'decir', 'voz', 'presidente', 'almuerzo', 'ahí', 'fujis

## **Libreria BOW - LDA: Gensim** 
---
Gensim es una biblioteca de código abierto para el modelado de temas sin supervisión y el procesamiento del lenguaje natural, que utiliza el aprendizaje automático estadístico moderno.

In [3]:
import pickle
import gensim
from gensim import corpora

# Diccionario de palabras del conjunto de datos
dicc = corpora.Dictionary(text_data)
# Creamos el Bag of Words por tendencia
corpus = [dicc.doc2bow(text) for text in text_data]

NUM = 8
# Entrenamos el modelo para 8 topicos
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM, id2word=dicc,random_state=100,update_every=1, passes=10,chunksize=100, alpha='auto')
ldamodel.save('modelo.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

lda = gensim.models.ldamodel.LdaModel.load('modelo.gensim')


(0, '0.009*"rodríguez" + 0.008*"alientadesdecaso" + 0.008*"liga1betssonxgolperu" + 0.008*"concha"')
(1, '0.003*"deber" + 0.003*"así" + 0.003*"poner" + 0.003*"pasar"')
(2, '0.003*"hacer" + 0.003*"seguir" + 0.003*"hoy" + 0.003*"sacar"')
(3, '0.003*"hacer" + 0.003*"pasar" + 0.003*"ahora" + 0.003*"solo"')
(4, '0.003*"ver" + 0.003*"dar" + 0.003*"hoy" + 0.003*"así"')
(5, '0.004*"poder" + 0.003*"mal" + 0.003*"ahí" + 0.003*"izquierda"')
(6, '0.003*"nombrar" + 0.003*"ver" + 0.003*"solo" + 0.003*"hacer"')
(7, '0.004*"hoy" + 0.004*"amar" + 0.004*"ser" + 0.003*"esperar"')


## **pyLDAvis** 
---
Libreria diseñada para ayudar a los usuarios a interpretar los temas en el topic modeling que se ha ajustado a un corpus. El paquete extrae información de el LDA ajustado para informar su visualización interactiva basada en la web.


In [4]:
!pip install pyLDAvis==2.1.2


In [5]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook(local=True)
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dicc)
pyLDAvis.display(lda_display)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## **Tópicos por Trend** 
---

In [6]:
def topic_documents(ldamodel=ldamodel, corpus=corpus, texts=dicc):
    # Creamos dataframe
    df_out = pd.DataFrame()
    # Obtenemos el tópico principal de cada documento
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Obtenemos el topico dominante, la contribución porcentual y las palabras clave de cada documento
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => Topico dominante
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                df_out = df_out.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    df_out.columns = ['Topico Dominante', 'Porcentaje de Contribución', 'KeyWords']
    # Agregamos texto original al final 
    contents = pd.Series(texts)
    df_out = pd.concat([df_out, contents], axis=1)
    return df_out

In [7]:
df_topicsXtrend = topic_documents(ldamodel=ldamodel, corpus=corpus, texts=dicc)

# Formato
df_dominant_topic = df_topicsXtrend.reset_index()
df_dominant_topic.columns = ['Trend', 'Topico Dominante', 'Porcentaje de Contribución', 'KeyWords', 'Tokens']
df_dominant_topic['Trend'] = df['Name']

# Mostramos
df_dominant_topic.head(39)

,Trend,Topico Dominante,Porcentaje de Contribución,KeyWords,Tokens
0,#ASUMETUVOTOIDIOTA,5.0,0.9984,"poder, mal, ahí, izquierda, pensar, presidente...",1ºministro
1,#FIGHT_OR_ESCAPE,7.0,0.9986,"hoy, amar, ser, esperar, have, buen, primero, ...",aburrir
2,#LALIGATUITERA,2.0,0.9955,"hacer, seguir, hoy, sacar, ganar, ver, buen, p...",afinar
3,#ONYOUR23RDMARK,7.0,0.9946,"hoy, amar, ser, esperar, have, buen, primero, ...",agarrar
4,#VACANCIAPEDROCASTILLO,1.0,0.9990,"deber, así, poner, pasar, hacer, partido, buen...",ahora
5,ADRIANA TUDELA,5.0,0.9958,"poder, mal, ahí, izquierda, pensar, presidente...",ahí
6,ALARCÓN,0.0,0.9972,"rodríguez, alientadesdecaso, liga1betssonxgolp...",almuerzo
7,ALDAIR,0.0,0.9893,"rodríguez, alientadesdecaso, liga1betssonxgolp...",andar
8,ALIANZA,2.0,0.9989,"hacer, seguir, hoy, sacar, ganar, ver, buen, p...",animal
9,ARLEY,0.0,0.9870,"rodríguez, alientadesdecaso, liga1betssonxgolp...",argumento
